In [1]:
import jsonlines
import pandas as pd
import numpy as np
import re
import spacy

In [2]:
resume = pd.read_csv ('cleaned_resume.csv')
resume

,Category,Cleaned_Resume
0,Data Science,skills programming languages python pandas num...
1,Data Science,education details may to may be uitrgpv data s...
2,Data Science,areas of interest deep learning control system...
3,Data Science,skills r python sap hana tableau sap hana sql ...
4,Data Science,education details mca ymcaust faridabad haryan...
...,...,...
957,Testing,computer skills proficient in ms office word b...
958,Testing,willingness to a ept the challenges positive ...
959,Testing,personal skills quick learner eagerness to lea...
960,Testing,computer skills software knowledge mspower poi...


In [3]:
nlp = spacy.load('en_core_web_md')

In [4]:
from spacy.pipeline import EntityRuler
ruler = nlp.add_pipe("entity_ruler", name="ruler", before="ner")
ruler.from_disk("jz_skill_patterns.jsonl")
nlp.pipe_names

['tok2vec',
 'tagger',
 'parser',
 'attribute_ruler',
 'lemmatizer',
 'ruler',
 'ner']

In [5]:
def get_skills(text):
    doc = nlp(text)
    myset = []
    subset = []
    for ent in doc.ents:
        if ent.label_ == "SKILL":
            subset.append(ent.text)
    myset.append(subset)
    return subset


def unique_skills(x):
    return list(set(x))

In [6]:
def clean(text):
    delete_n = re.sub(r'\\n', ' ', text)
    delete_x = re.sub(r'\\x[a-z0-9]+', ' ', delete_n)
    return delete_x

In [7]:
clean_text = []
for i in range(962):
    resume_clean = clean(resume['Cleaned_Resume'].loc[i])
    clean_text.append(resume_clean)

In [8]:
resume_skills = []
for i in range(962):
    resume_extract = unique_skills(get_skills(resume['Cleaned_Resume'].loc[i]))
    resume_skills.append(resume_extract)

In [9]:
resume['Skills'] = resume_skills
resume

,Category,Cleaned_Resume,Skills
0,Data Science,skills programming languages python pandas num...,"[flask, bootstrap, security, pandas, analytics..."
1,Data Science,education details may to may be uitrgpv data s...,"[keras, outlier, time series, python, ml, mach..."
2,Data Science,areas of interest deep learning control system...,"[flask, jupyter notebook, windows, software, d..."
3,Data Science,skills r python sap hana tableau sap hana sql ...,"[windows, deep learning, visual studio, specif..."
4,Data Science,education details mca ymcaust faridabad haryan...,"[data structure, python, data science, databas..."
...,...,...,...
957,Testing,computer skills proficient in ms office word b...,"[relay, diagram, windows, testing, engineering..."
958,Testing,willingness to a ept the challenges positive ...,"[windows, debugging, correctness, system testi..."
959,Testing,personal skills quick learner eagerness to lea...,"[certificate, functional testing, support, doc..."
960,Testing,computer skills software knowledge mspower poi...,"[software, latex, design]"


In [10]:
resume.to_csv('extract_skills.csv', index=False)